In [ ]:
from bs4 import BeautifulSoup
from pandas import json_normalize
import requests
import feedparser
import os

In [ ]:
train_rss_urls = {
    "tuoi_tre": ["https://tuoitre.vn/rss/tin-moi-nhat.rss", "https://tuoitre.vn/rss/thoi-su.rss", "https://tuoitre.vn/rss/the-gioi.rss", "https://tuoitre.vn/rss/phap-luat.rss", "https://tuoitre.vn/rss/kinh-doanh.rss", "https://tuoitre.vn/rss/nhip-song-so.rss", "https://tuoitre.vn/rss/xe.rss", "https://tuoitre.vn/rss/nhip-song-tre.rss", "https://tuoitre.vn/rss/van-hoa.rss", "https://tuoitre.vn/rss/giai-tri.rss", "https://tuoitre.vn/rss/the-thao.rss", "https://tuoitre.vn/rss/giao-duc.rss", "https://tuoitre.vn/rss/khoa-hoc.rss", "https://tuoitre.vn/rss/suc-khoe.rss", "https://tuoitre.vn/rss/gia-that.rss", "https://tuoitre.vn/rss/thu-gian.rss", "https://tuoitre.vn/rss/ban-doc-lam-bao.rss", "https://tuoitre.vn/rss/du-lich.rss"],
    "thanh_nien": ["https://thanhnien.vn/rss/home.rss", "https://thanhnien.vn/rss/thoi-su.rss", "https://thanhnien.vn/rss/chao-ngay-moi.rss", "https://thanhnien.vn/rss/the-gioi.rss", "https://thanhnien.vn/rss/kinh-te.rss", "https://thanhnien.vn/rss/doi-song.rss", "https://thanhnien.vn/rss/suc-khoe.rss", "https://thanhnien.vn/rss/gioi-tre.rss", "https://thanhnien.vn/rss/tieu-dung-thong-minh.rss", "https://thanhnien.vn/rss/giao-duc.rss", "https://thanhnien.vn/rss/du-lich.rss", "https://thanhnien.vn/rss/van-hoa.rss", "https://thanhnien.vn/rss/giai-tri.rss", "https://thanhnien.vn/rss/the-thao.rss", "https://thanhnien.vn/rss/cong-nghe-game.rss", "https://thanhnien.vn/rss/xe.rss", "https://thanhnien.vn/rss/thoi-trang-tre.rss", "https://thanhnien.vn/rss/ban-doc.rss", "https://thanhnien.vn/rss/video.rss", "https://thanhnien.vn/rss/dien-dan.rss", "https://thanhnien.vn/rss/podcast.rss", "https://thanhnien.vn/rss/nhat-ky-tet-viet.rss", "https://thanhnien.vn/rss/magazine.rss", "https://thanhnien.vn/rss/cung-con-di-tiep-cuoc-doi.rss", "https://thanhnien.vn/rss/ban-can-biet.rss", "https://thanhnien.vn/rss/blog-phong-vien.rss", "https://thanhnien.vn/rss/toi-viet.rss", "https://thanhnien.vn/rss/tno.rss", "https://thanhnien.vn/rss/tin-24h.rss", "https://thanhnien.vn/rss/thi-truong.rss", "https://thanhnien.vn/rss/tin-nhanh-360.rss"],
    "nld": ["https://nld.com.vn/rss/home.rss", "https://nld.com.vn/rss/thoi-su.rss", "https://nld.com.vn/rss/quoc-te.rss", "https://nld.com.vn/rss/lao-dong.rss", "https://nld.com.vn/rss/ban-doc.rss", "https://nld.com.vn/rss/phap-luat.rss", "https://nld.com.vn/rss/van-hoa-van-nghe.rss", "https://nld.com.vn/rss/giai-tri.rss", "https://nld.com.vn/rss/cong-nghe.rss", "https://nld.com.vn/rss/du-lich-xanh.rss", "https://nld.com.vn/rss/chuyen-trang-phu-nu.rss", "https://nld.com.vn/rss/gia-dinh.rss", "https://nld.com.vn/rss/dia-oc.rss", "https://nld.com.vn/rss/khoa-hoc.rss", "https://nld.com.vn/rss/du-lich.rss", "https://nld.com.vn/rss/moi-truong.rss", "https://nld.com.vn/rss/hoi-nong-dap-nhanh.rss", "https://nld.com.vn/rss/truy-vet-mang-xa-hoi.rss", "https://nld.com.vn/rss/noi-thang.rss", "https://nld.com.vn/rss/goc-nhin.rss", "https://nld.com.vn/rss/trich-dan-nong.rss", "https://nld.com.vn/rss/vung-mien.rss", "https://nld.com.vn/rss/doc-quyen.rss", "https://nld.com.vn/rss/mai-vang.rss", "https://nld.com.vn/rss/doi-song.rss", "https://nld.com.vn/rss/tu-hao-co-to-quoc.rss", "https://nld.com.vn/rss/bien-dao.rss", "https://nld.com.vn/rss/ly-tuong-song.rss"],
}

test_rss_urls = {
    "nld": ["https://nld.com.vn/rss/kinh-te.rss", "https://nld.com.vn/rss/suc-khoe.rss"],
}

val_rss_urls = {
    "nld": ["https://nld.com.vn/rss/giao-duc-khoa-hoc.rss", "https://nld.com.vn/rss/the-thao.rss"],
}

In [ ]:
def get_news_content(url):
    result = ""
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", "detail-content")
    if div is None:
        return ""
    texts = div.find_all("p")
    for text in texts:
        result += " " + text.get_text()
        
    return result

In [ ]:
def crawl_text(rss, path):
    if (not os.path.isdir(f"{path}")):
        os.mkdir(f"{path}")
        
    for article in rss:
        if (not os.path.isdir(f"{path}{article}")):
            os.mkdir(f"{path}{article}")
            
        print(article)

        index = 15501
        for rss_url in rss[article]:
            print(rss_url)

            news_feed = feedparser.parse(rss_url)
            df_news_feed = json_normalize(news_feed.entries)
            
            if "summary" in df_news_feed.keys():
                for text, news_url in zip(df_news_feed.summary, df_news_feed.link):
                    soup = BeautifulSoup(text, "html.parser")
                    print(news_url)
                    with open(os.path.join(f"{path}{article}", f'{index}.txt'), 'w', encoding='utf-8') as f:
                        try:
                            f.write(soup.get_text() + " " + get_news_content(news_url))
                        except:
                            continue
                        # f.write(soup.get_text())
                    index += 1

In [ ]:
crawl_text(train_rss_urls, "./temp/")
crawl_text(test_rss_urls, "./test_set/")
crawl_text(val_rss_urls, "./val_set/")